In [1]:
pip install transformers datasets sentencepiece evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvi

In [2]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import pandas as pd

df = pd.read_csv("/kaggle/input/youtube-transcripts-and-summaries/youtube_transcripts_with_summaries.csv")
# df.head()

In [4]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/kaggle/input/youtube-transcripts-and-summaries/youtube_transcripts_with_summaries.csv")

# Rename columns  
dataset = dataset.rename_column("transcript", "article")
dataset = dataset.rename_column("summary", "highlights")

# Split into train and eval
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'title', 'news_channel', 'video_id', 'article', 'highlights'],
        num_rows: 396
    })
    test: Dataset({
        features: ['url', 'title', 'news_channel', 'video_id', 'article', 'highlights'],
        num_rows: 45
    })
})

In [5]:
dataset['test'][0]["highlights"]

'Bengaluru is facing heavy rains and flooding again, frustrating residents as the problem recurs yearly without solutions. Locals have turned to humor online, creating memes to cope. Meanwhile, news outlet First Post pledges accurate, verified reporting—owning past mistakes and promising transparency amid the noise of breaking news and misinformation.'

In [6]:
# dataset_samsum = load_dataset('samsum')
# dataset_samsum

In [7]:
# pipe_out = pipe(dataset_samsum['test'][0]["dialogue"])
# print('Summary')
# print(pipe_out[0]['summary_text'].replace(". <n>", ".\n"))

In [8]:
# from transformers import pipeline, set_seed

# pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
# pipe_out = pipe(dataset["test"][0]["article"])
# print(pipe_out[0]["summary_text"].replace(". <n>", ".\n"))

In [9]:
from transformers import PegasusTokenizer
import torch

model_name = "google/pegasus-cnn_dailymail"
tokenizer = PegasusTokenizer.from_pretrained(model_name)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

In [10]:
missing_indices = [i for i, val in enumerate(dataset['train']['highlights']) if val is None or val.strip() == ""]
print(f"Missing entries at indices: {missing_indices}")


Missing entries at indices: []


In [11]:
dataset['train'][351]['video_id']

'ddtZjbRno-Q'

In [12]:
# def convert_examples_to_features(example_batch):
#     # print(f"example_batch: {example_batch['article']}")
#     input_encodings = tokenizer(example_batch['article'], max_length=1024, truncation=True, padding="max_length")

#     with tokenizer.as_target_tokenizer():
#         target_encodings = tokenizer(example_batch['highlights'], max_length=128, truncation=True, padding="max_length")

#     return {"input_ids": input_encodings['input_ids'],
#             "attention_mask": input_encodings['attention_mask'],
#            "labels": target_encodings['input_ids']}

max_input_length = 1024
max_target_length = 128

# def tokenize_function(examples):
#     model_inputs = tokenizer(
#         text=examples["article"],
#         max_length=max_input_length,
#         truncation=True,
#         padding="max_length"
#     )
#     with tokenizer.as_target_tokenizer():  # still works for now, but not future-proof
#         labels = tokenizer(
#             text_target=examples["highlights"],
#             max_length=max_target_length,
#             truncation=True,
#             padding="max_length"
#         )
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

def tokenize_function(examples):
    model_inputs = tokenizer(
        text=examples["article"],
        max_length=1024,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        text_target=examples["highlights"],    # target (used as labels)
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function)

Map:   0%|          | 0/396 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

In [13]:
from transformers import PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

2025-06-30 14:16:23.034519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751292983.225698      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751292983.290502      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [14]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./pegasus-finetune-news",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
    push_to_hub=True,
    fp16=False
)



In [15]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5ebb9972096fe9e9fdebc926bc50428f8491e97a2d9c4ad625d23843100431e1
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [16]:
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in labels as padding token
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 2) for k, v in result.items()}
    return result

In [ ]:
from huggingface_hub import login
login(token="api_token")

In [ ]:
import wandb
wandb.login(key="api_token")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ugochiibekwe13 (ugochiibekwe13-federal-university-of-technology-owerri) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [19]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipykernel_35/4240050231.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,6.592600,6.343072,34.770000,11.000000,23.630000,23.630000
2,6.182800,5.873182,36.330000,12.160000,24.510000,24.510000
3,5.609700,5.218021,38.440000,13.110000,25.830000,25.780000
4,4.818100,4.387918,38.560000,12.790000,26.360000,26.310000
5,4.554700,4.004547,38.750000,12.770000,26.190000,26.190000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=990, training_loss=5.793119133843316, metrics={'train_runtime': 2217.045, 'train_samples_per_second': 0.893, 'train_steps_per_second': 0.447, 'total_flos': 5721139540131840.0, 'train_loss': 5.793119133843316, 'epoch': 5.0})

In [ ]:
# Result of the Training
# Epoch	Training Loss	Validation Loss	Rouge1	Rouge2	Rougel	Rougelsum
# 1	6.592600	6.343072	34.770000	11.000000	23.630000	23.630000
# 2	6.182800	5.873182	36.330000	12.160000	24.510000	24.510000
# 3	5.609700	5.218021	38.440000	13.110000	25.830000	25.780000
# 4	4.818100	4.387918	38.560000	12.790000	26.360000	26.310000
# 5	4.554700	4.004547	38.750000	12.770000	26.190000	26.190000


In [20]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/ibk007/pegasus-finetune-news/commit/8c3bd5a3c959236f094b22bbc7cd4379948c7cd3', commit_message='End of training', commit_description='', oid='8c3bd5a3c959236f094b22bbc7cd4379948c7cd3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ibk007/pegasus-finetune-news', endpoint='https://huggingface.co', repo_type='model', repo_id='ibk007/pegasus-finetune-news'), pr_revision=None, pr_num=None)

In [21]:
eval_results = trainer.evaluate()
eval_results

{'eval_loss': 4.004547119140625,
 'eval_rouge1': 38.75,
 'eval_rouge2': 12.77,
 'eval_rougeL': 26.19,
 'eval_rougeLsum': 26.19,
 'eval_runtime': 97.2002,
 'eval_samples_per_second': 0.463,
 'eval_steps_per_second': 0.463,
 'epoch': 5.0}

In [ ]:
# eval_results
# {'eval_loss': 4.004547119140625,
#  'eval_rouge1': 38.75,
#  'eval_rouge2': 12.77,
#  'eval_rougeL': 26.19,
#  'eval_rougeLsum': 26.19,
#  'eval_runtime': 97.2002,
#  'eval_samples_per_second': 0.463,
#  'eval_steps_per_second': 0.463,
#  'epoch': 5.0
# }

In [22]:
import pandas as pd

df = pd.DataFrame([eval_results])
print(df.to_string(index=False))

 eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  eval_runtime  eval_samples_per_second  eval_steps_per_second  epoch
  4.004547        38.75        12.77        26.19           26.19       97.2002                    0.463                  0.463    5.0


In [25]:
!pip install youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.7/485.7 kB 9.4 MB/s eta 0:00:00:00:01


In [26]:
import pandas as pd
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound, VideoUnavailable


def fetch_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        full_text = " ".join([entry['text'] for entry in transcript])
        return full_text
    except (TranscriptsDisabled, NoTranscriptFound, VideoUnavailable):
        raise
    except Exception as e:
        raise

In [27]:
# https://youtu.be/SxSS1UJtEAc?si=POQ72a1XaBATUXhV
# https://www.youtube.com/watch?v=hshUpii9uqY

transcript = fetch_transcript("SxSS1UJtEAc")
transcript

'Remember when Donald Trump and Elon Musk were joined at the hip? It was cute while it lasted, you could say. Together, the two the two of them had it all. Money, power, and big egos. Musk bankrolled Trump\'s election campaign. He gave him more than $250 million. Trump returned the favor with VIP access to the White House. He even created a government department for Musk. It was called Doge, the Department of Government Efficiency. Trump called Musk a visionary. Musk called Trump a friend. Like I said, it was a bromance for the ages. But like most great love stories, this one was not built to last. Elon Musk is now criticizing Donald Trump\'s politics. You see, last week the US Congress approved a bill, Trump\'s new tax bill. He calls it big and beautiful. That\'s what he calls the tax bill, big and beautiful. It includes million-doll tax breaks, increased defense spending, and funding for a border wall. The bill is heading to the US Senate. Once it is passed by the Senate, it will mas

In [28]:
from transformers import pipeline

summarizer = pipeline("summarization", model="ibk007/pegasus-finetune-news")
output = summarizer(transcript)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [29]:
output

/*
[{'summary_text': "Elon Musk and Donald Trump's bromance is ending, with Musk criticizing the president's new tax bill. He says it increases the deficit and undermines his government's efficiency. Both Trump and Musk insist it's not a breakup, but it's clear this was inevitable. In Trump's world, loyalty is a one-way street. Don't get too comfortable. One minute you're flying to state dinners, the next you're out. You're only useful until you're not."}]
*/

[{'summary_text': "Elon Musk and Donald Trump's bromance is ending, with Musk criticizing the president's new tax bill. He says it increases the deficit and undermines his government's efficiency. Both Trump and Musk insist it's not a breakup, but it's clear this was inevitable. In Trump's world, loyalty is a one-way street. Don't get too comfortable. One minute you're flying to state dinners, the next you're out. You're only useful until you're not."}]